In [3]:
import os
import pathlib
import re
import copy
import warnings
import time
import json
import pickle

from sklearn import svm
from sys import getsizeof

import cv2
import xmltodict
import numpy as np
import pandas as pd
import tiffcapture as tc

from tifffile import imsave
from matplotlib import pyplot as pt

%matplotlib inline

# I. Unsynchronized

In [4]:
file = open("../results/unsyn_log.txt", "r")
log_unsyn = file.readlines()
file.close()

In [42]:
log_unsyn

['Processing position 01...\n',
 '  Reading files\n',
 '  Reading files done. Time elapsed: 0.228626\n',
 '  Parsing trees and values\n',
 '  Parsing done. Time elapsed: 141.189137\n',
 '  Filtering tree\n',
 '  Filtering done. Time elapsed: 0.171749\n',
 '  Getting contour\n',
 '    # number of detected slits: 305\n',
 '    # number of occuped slits: 68\n',
 '    # number of slits with doubly-placed trees: 12\n',
 '    # number of slits with triply-placed trees: 1\n',
 '    # number of slits with quadruply-placed trees: 0\n',
 '    # number of trees: 164\n',
 '    # of singly-placed trees with 0 div: 36\n',
 '    # of singly-placed trees with 0 div with last timestamp < t(treatment): 21\n',
 '    # of singly-placed trees with 1 div: 9\n',
 '    # of singly-placed trees with >1 divs: 10\n',
 '    # trees with division time before treatment: 8\n',
 "    # trees with both branches' last measurement time after treatment: 0\n",
 '  Contour recognition done. Time elapsed: 0.145084\n',
 '  E

In [43]:
positions = list(filter(lambda x: "Processing position" in x, log_unsyn))
positions = [re.search("[0-9]+", x).group(0) for x in positions]
positions = [int(n) for n in positions]

In [52]:
detected_slits = list(filter(lambda x: "number of detected slits" in x, log_unsyn))
detected_slits = [re.search("number of detected slits: ([0-9]+)", x).group(1) for x in detected_slits]
detected_slits = [int(n) for n in detected_slits]

In [53]:
occupied_slits = list(filter(lambda x: "number of occuped slits" in x, log_unsyn))
occupied_slits = [re.search("number of occuped slits: ([0-9]+)", x).group(1) for x in occupied_slits]
occupied_slits = [int(n) for n in occupied_slits]

In [54]:
doubly_placed_slits = list(filter(lambda x: "number of slits with doubly-placed trees" in x, log_unsyn))
doubly_placed_slits = [re.search("number of slits with doubly-placed trees: ([0-9]+)", x).group(1) for x in doubly_placed_slits]
doubly_placed_slits = [int(n) for n in doubly_placed_slits]

In [55]:
triply_placed_slits = list(filter(lambda x: "number of slits with triply-placed trees" in x, log_unsyn))
triply_placed_slits = [re.search("number of slits with triply-placed trees: ([0-9]+)", x).group(1) for x in triply_placed_slits]
triply_placed_slits = [int(n) for n in triply_placed_slits]

In [56]:
num_trees = list(filter(lambda x: "number of trees" in x, log_unsyn))
num_trees =  [re.search("number of trees: ([0-9]+)", x).group(1) for x in num_trees]
num_trees = [int(n) for n in num_trees]

In [44]:
no_div = list(filter(lambda x: "singly-placed trees with 0 div:" in x, log_unsyn))
no_div = [re.search("trees with 0 div: ([0-9]+)", x).group(1) for x in no_div]
no_div = [int(n) for n in no_div]

In [45]:
no_div_dropped = list(filter(lambda x: "singly-placed trees with 0 div with last timestamp < t(treatment)" in x, log_unsyn))
no_div_dropped = [re.search("t\(treatment\): ([0-9]+)", x).group(1) for x in no_div_dropped]
no_div_dropped = [int(n) for n in no_div_dropped]

In [57]:
one_div = list(filter(lambda x: "singly-placed trees with 1 div" in x, log_unsyn))
one_div = [re.search("trees with 1 div: ([0-9]+)", x).group(1) for x in one_div]
one_div = [int(n) for n in one_div]

In [47]:
morethan_one_div = list(filter(lambda x: "of singly-placed trees with >1 divs" in x, log_unsyn))
morethan_one_div = [re.search("trees with >1 divs: ([0-9]+)", x).group(1) for x in morethan_one_div]
morethan_one_div = [int(n) for n in morethan_one_div]

In [49]:
div_before_treatment = list(filter(lambda x: "trees with division time before treatment" in x, log_unsyn))
div_before_treatment = [re.search("division time before treatment: ([0-9]+)", x).group(1) for x in div_before_treatment]
div_before_treatment = [int(n) for n in div_before_treatment]

In [50]:
death_after_treatment = list(filter(lambda x: "trees with both branches' last measurement" in x, log_unsyn))
death_after_treatment = [re.search("measurement time after treatment: ([0-9]+)", x).group(1) for x in death_after_treatment]
death_after_treatment = [int(n) for n in death_after_treatment]

In [58]:
singly_placed_slits = list(np.array(no_div) + np.array(one_div) + np.array(morethan_one_div))

In [59]:
higher_placed_slits = list(np.array(occupied_slits) - 
                           np.array(singly_placed_slits) - 
                           np.array(doubly_placed_slits) - 
                           np.array(triply_placed_slits))

In [61]:
stats = pd.DataFrame({'POSITION':positions,
                      'Detected slits': detected_slits,
                      'Occupied slits': occupied_slits,
                      'Singly-placed slits': singly_placed_slits,
                      'Doubly-placed slits': doubly_placed_slits,
                      'Triply-placed slits': triply_placed_slits,
                      'Higher-placed slits': higher_placed_slits,
                      'Number of trees': num_trees,
                      'Singly-placed trees 0 div': no_div,
                      'Singly-placed trees 0 div and T(last_time) < T(treatment) [DROPPED]': no_div_dropped,
                      'Singly-placed trees >1 divs': morethan_one_div,
                      'Singly-placed trees 1 div': one_div,
                      'Singly-placed trees T(div) < T(treatment)': div_before_treatment,
                      'Singly-placed trees T(end_of_track) > T(treatment)': death_after_treatment
                })

stats = stats[['POSITION', 'Detected slits', 'Occupied slits', 'Singly-placed slits', 
               'Doubly-placed slits', 'Triply-placed slits', 'Higher-placed slits',
               'Number of trees', 'Singly-placed trees 0 div',
               'Singly-placed trees 0 div and T(last_time) < T(treatment) [DROPPED]', 'Singly-placed trees >1 divs',
               'Singly-placed trees 1 div', 'Singly-placed trees T(div) < T(treatment)',
               'Singly-placed trees T(end_of_track) > T(treatment)'
              ]]

stats

,POSITION,Detected slits,Occupied slits,Singly-placed slits,Doubly-placed slits,Triply-placed slits,Higher-placed slits,Number of trees,Singly-placed trees 0 div,Singly-placed trees 0 div and T(last_time) < T(treatment) [DROPPED],Singly-placed trees >1 divs,Singly-placed trees 1 div,Singly-placed trees T(div) < T(treatment),Singly-placed trees T(end_of_track) > T(treatment)
0,1,305,68,55,12,1,0,164,36,21,10,9,8,0
1,2,314,57,52,5,0,0,124,37,14,4,11,7,3
2,3,291,57,48,8,1,0,134,35,30,3,10,10,0
3,6,300,76,64,11,1,0,178,27,9,18,19,15,8
4,7,313,62,47,14,0,1,160,15,6,24,8,8,1
5,8,310,78,69,9,0,0,174,38,18,10,21,21,8
6,9,305,77,59,16,2,0,194,32,23,20,7,6,2
7,10,303,54,45,9,0,0,126,21,14,12,12,9,2
8,12,253,79,60,17,2,0,200,26,18,20,14,12,6
9,14,234,79,56,13,7,3,232,25,14,17,14,14,7


In [62]:
stats.to_csv('../results/stats_unsyn.csv')

# II. Synchronized

In [4]:
file = open("../results/syn_log.txt", "r")
log_syn = file.readlines()
file.close()

In [5]:
log_syn

['Processing position 01...\n',
 '  Reading files\n',
 '  Reading files done. Time elapsed: 0.247329\n',
 '  Parsing trees and values\n',
 '  Parsing done. Time elapsed: 38.788574\n',
 '  Filtering tree\n',
 '  Filtering done. Time elapsed: 0.059220\n',
 '  Getting contour\n',
 '    # number of detected slits: 294\n',
 '    # number of occupied slits: 40\n',
 '    # number of slits with singly-placed tree: 76\n',
 '    # number of slits with doubly-placed trees: 4\n',
 '    # number of slits with triply-placed trees: 0\n',
 '    # number of slits with quadruply-placed trees: 0\n',
 '    # number of trees: 84\n',
 '    # of singly- and doubly-placed trees with 0 div: 38\n',
 '    # of singly- and doubly-placed trees with 0 div with last timestamp < t(treatment): 33\n',
 '    # of singly- and doubly-placed trees with 1 div: 1\n',
 '    # of singly- and doubly-placed trees with >1 divs: 1\n',
 '    # trees with division time before treatment: 1\n',
 "    # trees with both branches' last m

In [6]:
positions = list(filter(lambda x: "Processing position" in x, log_syn))
positions = [re.search("[0-9]+", x).group(0) for x in positions]
positions = [int(n) for n in positions]

In [7]:
detected_slits = list(filter(lambda x: "number of detected slits" in x, log_syn))
detected_slits = [re.search("number of detected slits: ([0-9]+)", x).group(1) for x in detected_slits]
detected_slits = [int(n) for n in detected_slits]

In [8]:
occupied_slits = list(filter(lambda x: "number of occupied slits" in x, log_syn))
occupied_slits = [re.search("number of occupied slits: ([0-9]+)", x).group(1) for x in occupied_slits]
occupied_slits = [int(n) for n in occupied_slits]

In [13]:
singly_placed_slits = list(filter(lambda x: "number of slits with singly-placed tree" in x, log_syn))
singly_placed_slits = [re.search("number of slits with singly-placed tree: ([0-9]+)", x).group(1) for x in singly_placed_slits]
singly_placed_slits = [int(n) for n in singly_placed_slits]
singly_placed_slits = [int(n / 2) for n in singly_placed_slits]

In [14]:
doubly_placed_slits = list(filter(lambda x: "number of slits with doubly-placed trees" in x, log_syn))
doubly_placed_slits = [re.search("number of slits with doubly-placed trees: ([0-9]+)", x).group(1) for x in doubly_placed_slits]
doubly_placed_slits = [int(n) for n in doubly_placed_slits]
doubly_placed_slits = [int(n / 2) for n in doubly_placed_slits]

In [15]:
triply_placed_slits = list(filter(lambda x: "number of slits with triply-placed trees" in x, log_syn))
triply_placed_slits = [re.search("number of slits with triply-placed trees: ([0-9]+)", x).group(1) for x in triply_placed_slits]
triply_placed_slits = [int(n) for n in triply_placed_slits]
triply_placed_slits = [int(n / 2) for n in triply_placed_slits]

In [16]:
higher_placed_slits = list(np.array(occupied_slits) - 
                           np.array(singly_placed_slits) - 
                           np.array(doubly_placed_slits) - 
                           np.array(triply_placed_slits))

In [17]:
num_trees = list(filter(lambda x: "number of trees" in x, log_syn))
num_trees =  [re.search("number of trees: ([0-9]+)", x).group(1) for x in num_trees]
num_trees = [int(n) for n in num_trees]

In [18]:
no_div = list(filter(lambda x: "of singly- and doubly-placed trees with 0 div:" in x, log_syn))
no_div = [re.search("of singly- and doubly-placed trees with 0 div: ([0-9]+)", x).group(1) for x in no_div]
no_div = [int(n) for n in no_div]

In [19]:
stats = pd.DataFrame({'POSITION':positions,
                      'Detected slits': detected_slits,
                      'Occupied slits': occupied_slits,
                      'Singly-placed slits': singly_placed_slits,
                      'Doubly-placed slits': doubly_placed_slits,
                      'Triply-placed slits': triply_placed_slits,
                      'Higher-placed slits': higher_placed_slits,
                      'Number of trees': num_trees,
                      'Singly-placed trees 0 div': no_div
                })

stats = stats[['POSITION', 'Detected slits', 'Occupied slits', 'Singly-placed slits', 
               'Doubly-placed slits', 'Triply-placed slits', 'Higher-placed slits',
               'Number of trees', 'Singly-placed trees 0 div'
              ]]

stats

,POSITION,Detected slits,Occupied slits,Singly-placed slits,Doubly-placed slits,Triply-placed slits,Higher-placed slits,Number of trees,Singly-placed trees 0 div
0,1,294,40,38,2,0,0,84,38
1,2,305,31,30,1,0,0,64,29
2,3,310,66,51,14,1,0,164,50
3,6,311,26,24,2,0,0,56,25
4,7,313,16,16,0,0,0,32,16
5,8,280,70,63,7,0,0,154,62
6,9,299,31,29,2,0,0,66,31
7,11,290,50,46,3,1,0,110,47
8,13,310,76,63,12,1,0,180,69
9,15,263,32,27,4,1,0,76,29


In [21]:
stats.to_csv('../results/stats_syn.csv')